In [2]:
!pip install -qU google-generativeai


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [71]:
import json

import google.generativeai as genai

with open('secret.json') as f:
    secret = json.load(f)

genai.configure(api_key=secret['GEMINI_TOKEN'])

import requests

def get_website_content(url: str) -> str:
  """Get the content from a specific URL similar to curl -X GET.
  
  Args.
      url (str, required): The target URL is either remote or local.
  
  Returns.
      str: The raw content from a specific URL.
  """
  response = requests.get(url)
  return response.text

def post_request(url: str, data: str) -> dict:
  """
  Send a request similar to curl -X POST.

  Args.
      url (str, required): Target URL.
      data (str, required): The data of the form in "id=value1&name=value2&description=value3" format.

  Returns.
      dict: Dictionary containing the status code and response content.
  """
  response = requests.post(url, headers={"Content-Type": "application/x-www-form-urlencoded"}, data=data)
  
  try:
      response_content = response.json()
  except ValueError:
      response_content = response.text

  return {
      "status_code": response.status_code,
      "content": response_content,
  }

system_instruction = '''You are a professional web crawler and familiar with swagger usage.
You can get the content you want by yourself through web api, 
and when you utilize web api, you will actively list which api is used.
If you don't know the answer to a question, just answer no, don't make up an answer.
'''

model = genai.GenerativeModel(model_name='gemini-1.5-pro',
                              tools=[get_website_content, post_request],
                              system_instruction=system_instruction,)

model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x10c756310>,
    system_instruction="You are a professional web crawler and familiar with swagger usage.\nYou can get the content you want by yourself through web api, \nand when you utilize web api, you will actively list which api is used.\nIf you don't know the answer to a question, just answer no, don't make up an answer.\n",
    cached_content=None
)

In [72]:
chat_session = model.start_chat(
    enable_automatic_function_calling=True,
        history=[
        {
            "role": "user",
            "parts": [
                "I have a website and this website provides full swagger: http://localhost:50000/apispec_1.json Please tell me the features of this website."
            ]
        },
        {
            "role": "model",
            "parts": [
                """This website provides four API endpoints:

- Create or update a product: You can send a POST request to `/api/product` with product ID, name, and description to create or update a product.
- Delete a product: You can send a POST request to `/api/product/delete/{product_id}` to delete a product by its ID.
- Get a product: You can send a GET request to `/api/product/{product_id}` to retrieve the details of a product by its ID.
- Get all products: You can send a GET request to `/api/products` to get a list of all products. """
            ]
        },
        {
            "role": "user",
            "parts": [
                "List all the products and find out for yourself if they are fruit products from the description."
            ]
        },
        {
            "role": "model",
            "parts": [
                """Based on the product descriptions, the following products are fruits:

* **Apple (ID: 123)**: "Fruit" """
            ]
        },
        {
            "role": "user",
            "parts": [
                "Find all animals, and the description should contain animal."
            ]
        },
        {
            "role": "model",
            "parts": [
                """Based on the product descriptions, the following products are animals:

* **Bird (ID: 234)**: "Animal"
* **Cat (ID: 345)**: "Animal" """
            ]
        },
        {
            "role": "user",
            "parts": [
                "Create a new product by api based on all existing product informantion."
            ]
        },
        {
            "role": "model",
            "parts": [
                """I have created a new product with the following information:

* **ID:** 456
* **Name:** All-in-one
* **Description:** This is a product contains animals and fruits. """
            ]
        }
    ]
)

#response = chat_session.send_message("Change the name of the product whose ID is 456 to Dodge, and of course change the description to the vehicle brand.")
response = chat_session.send_message("How would you change the product name and description? Do not post a real request, just tell me which function and what parameters.")

print(response.text)

I0000 00:00:1721890744.003208 34148132 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


I can use the `post_request` function with the following parameters to change the product name and description:

```python
print(default_api.post_request(url='http://localhost:50000/api/product', data='id=456&name=New%20Name&description=New%20Description'))
``` 



In [74]:
response = chat_session.send_message("Change the name of the product whose ID is 456 to Dodge, and change the description to the vehicle brand.")

print(response.text)

OK. I've updated the product with ID 456. 



In [76]:
response = chat_session.send_message("Create a new product by your own thought")

print(response.text)

I have created a new product with the following information:

* **ID:** 567
* **Name:** AI Power
* **Description:** This is the product that can change the world 

